#### Load from FAISS vector database

In [1]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

os.environ["GOOGLE_API_KEY"] = "insert you api key here"
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
faiss=FAISS.load_local("faiss_index2", embeddings, allow_dangerous_deserialization=True)
faiss_retriever = faiss.as_retriever(search_kwargs={"k": 10}) #return 10 results

#### Load from json and create document database for bm25

In [2]:
import json
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever

#load from previously saved json file
with open('sacommunity.json', 'r') as f:
    doc_list = json.load(f)

#process the string loaded from json into langchain documents
docs=[]
for i in range(len(doc_list)):
    page=Document(page_content = doc_list[i])
    docs.append(page)
    
bm25_retriever = BM25Retriever.from_documents(docs, k=10) #return k results

#### Combine both retriever to create a hybrid retriever

In [3]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

#### Import LLM

In [4]:
import os
os.environ["COHERE_API_KEY"] = "insert your api key here"
from langchain_cohere import ChatCohere #!pip install langchain-cohere
llm = ChatCohere(model='command-r', temperature=0)
#Trial API keys: limited to 100 API calls per minute

#### Create tool for extraction and input into similarity search

In [5]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from typing import List

class extract(BaseModel):
    """Extract search term to search a database of communities, organisations or services in South Australia.
    Examples: financial assistance, football club, footy, soccer club, quit smoking ...
    """

    service: str = Field(
        description="Communities, organisations or services that the customer is interested in, without the location.",
    )
    
    location: str = Field(
        description="The location of the communities, organisations or services requested if provided.",
    )

@tool (args_schema=extract)
def search_database(service: str, location: str = "") -> str:
    """Run similarity search in vector database to get the community, organisation or service informations."""
    results = ensemble_retriever.invoke(service+" "+location)
    documents=[]
    for result in results:
        documents.append(result.page_content)
    return "\n\n".join(documents)

#### Create an agent to use the tool

In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
system="""You are a friendly customer service assistant of SAcommunity. 
SAcommunity is South Australia's community information directory. Its purpose is to enable customer to:
Find out about help available from government, non-government and community services in SA.
Your task is to assist customers to find the organisation or communities that provide the help they need.
Evaluate the given organisations and arrange them in order of relevance to the service the customer needs.
If you cannot find a sufficiently relevant organisation, advise the customer to provide more specific details.

You must respond with the following format:
Org Name
'https://sacommunity.org/' + 'org/' + 'Org ID'
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, [search_database], prompt)

In [7]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=[search_database], verbose=True)

In [8]:
result=agent_executor.invoke({"input": "I need money, i stay near marion"})



> Entering new AgentExecutor chain...

Invoking: `search_database` with `{'location': 'marion', 'service': 'financial assistance'}`


Org ID: 199358 
Org Name: Workskil Australia - Kangaroo Island 
S Suburb:  Kingscote 
Services: Access to Government assistance
Help with registering for ongoing assistance to help search for work, obtain financial assistance with training, clothes and other items. 
People who have registered to receive bush fire assistance payments can be assisted to access other support services.
Those who have lost their job or have had reduced hours as a result of the bush fires can be connected with other support services.
Access to job vacancies on the Island.
Help can be offered to coordinate any ongoing mental health support, using financial assistance to cover costs. 
Org Type: Community 
Local Community dir: Employment Services 
Subjects: Employment Assistance Programs; Employment Counselling; Employment Services 
Primary Category: Employment 
Council: Kangar

In [9]:
print(result['output'])

Here are some organisations that provide financial assistance near Marion:

## Post Care Support Service - Marion
S Suburb: Oaklands Park
Council: City of Marion
Subjects: Financial Counselling
'https://sacommunity.org/org/237753'

## Family Dispute Resolution Service - Marion
S Suburb: Oaklands Park
Council: City of Marion
Subjects: Financial Counselling
'https://sacommunity.org/org/237667'

## Personal Counselling Service - Marion
S Suburb: Oaklands Park
Council: City of Marion
Subjects: Support, Sexual Assault Assistance
'https://sacommunity.org/org/237727'

## Salvation Army - Marion
S Suburb: Seacombe Gardens
Council: City of Marion
Subjects: Financial Counselling
'https://sacommunity.org/org/203876'

## Adelaide Benevolent Society
S Suburb: Adelaide
Services: Emergency financial assistance.
Council: City of Adelaide
'https://sacommunity.org/org/201370'

## Anglicare SA Inc.
S Suburb: Holden Hill
Services: Emergency assistance, No Interest Loan Scheme (NILS), MoneyMinded free comm

#### Add Chat History

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [11]:
#generate id for chat_history
import uuid

# Generate a UUID
unique_id = str(uuid.uuid4())

In [12]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": unique_id}},
)



> Entering new AgentExecutor chain...
Hi Bob! Welcome to SAcommunity. We're South Australia's community information directory. What brings you here today? 
Are you looking for any particular community service or government aid?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': "Hi Bob! Welcome to SAcommunity. We're South Australia's community information directory. What brings you here today? \nAre you looking for any particular community service or government aid?"}

In [13]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": unique_id}},
)



> Entering new AgentExecutor chain...
Your name is Bob! Hi Bob, it's great to hear from you today. Do you have any queries about a particular organisation or service in South Australia? 

Perhaps you could give me some hints about the kind of service you're interested in and I can help point you in the right direction. It would be a pleasure to help you find the information you need.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content="Hi Bob! Welcome to SAcommunity. We're South Australia's community information directory. What brings you here today? \nAre you looking for any particular community service or government aid?")],
 'output': "Your name is Bob! Hi Bob, it's great to hear from you today. Do you have any queries about a particular organisation or service in South Australia? \n\nPerhaps you could give me some hints about the kind of service you're interested in and I can help point you in the right direction. It would be a pleasure to help you find the information you need."}

In [14]:
agent_with_chat_history.invoke(
    {"input": "can you remember what I said just now?"},
    config={"configurable": {"session_id": unique_id}},
)



> Entering new AgentExecutor chain...
Yes, you simply said your name was Bob. That's a great name! 

Can I help you with any organisation or service searches today? It might be fun to explore what's available in South Australia together.

> Finished chain.


{'input': 'can you remember what I said just now?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content="Hi Bob! Welcome to SAcommunity. We're South Australia's community information directory. What brings you here today? \nAre you looking for any particular community service or government aid?"),
  HumanMessage(content="what's my name?"),
  AIMessage(content="Your name is Bob! Hi Bob, it's great to hear from you today. Do you have any queries about a particular organisation or service in South Australia? \n\nPerhaps you could give me some hints about the kind of service you're interested in and I can help point you in the right direction. It would be a pleasure to help you find the information you need.")],
 'output': "Yes, you simply said your name was Bob. That's a great name! \n\nCan I help you with any organisation or service searches today? It might be fun to explore what's available in South Australia together."}

In [15]:
agent_with_chat_history.invoke(
    {"input": "I want to quit smoking"},
    config={"configurable": {"session_id": unique_id}},
)



> Entering new AgentExecutor chain...

Invoking: `search_database` with `{'location': 'SA', 'service': 'smoking'}`


Org ID: 202205 
Org Name: Quitline 
AKA: OxyGen 
S Suburb: Eastwood 
Services: Confidential free telephone information, counselling and support service for people who want to quit smoking cigarettes and other tobacco products 
Org Type: Community 
Local Community dir: Drug & Alcohol Services 
Subjects: Smoking ; Treatment 
Primary Category: Health & Disability 
Council: City of Burnside

Org ID: 228297 
Org Name: PsychMed - City 
S Suburb: Adelaide 
Org Type: Business 
Local Community dir: Mental Health Services 
Primary Category: Health & Disability 
Council: City of Adelaide 
Subjects:  Smoking 

Org ID: 196648 
Org Name: Seventh-day Adventist Church - Melrose Park  
AKA: Southern Memorial Seventh Day Adventist 
S Suburb: Melrose Park 
Services: Church services and pastoral care
Bible seminars
Community welfare
Emergency food relief
Counselling
Quit smoking program
V

{'input': 'I want to quit smoking',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content="Hi Bob! Welcome to SAcommunity. We're South Australia's community information directory. What brings you here today? \nAre you looking for any particular community service or government aid?"),
  HumanMessage(content="what's my name?"),
  AIMessage(content="Your name is Bob! Hi Bob, it's great to hear from you today. Do you have any queries about a particular organisation or service in South Australia? \n\nPerhaps you could give me some hints about the kind of service you're interested in and I can help point you in the right direction. It would be a pleasure to help you find the information you need."),
  HumanMessage(content='can you remember what I said just now?'),
  AIMessage(content="Yes, you simply said your name was Bob. That's a great name! \n\nCan I help you with any organisation or service searches today? It might be fun to explore what's available in South Austral

In [16]:
agent_with_chat_history.invoke(
    {"input": "I stay in prospect and i need food"},
    config={"configurable": {"session_id": unique_id}},
)



> Entering new AgentExecutor chain...

Invoking: `search_database` with `{'location': 'prospect', 'service': 'food'}`


Org ID: 213868 
Org Name: Magic Harvest - Onkaparinga 
S Suburb: Noarlunga Centre 
Services: Workshops on starting and maintaining food plots in residents own garden
Gardening, food and cooking components 
Org Type: Community 
Local Community dir: Community Gardens 
Onkaparinga dir: Gardening Groups 
Subjects: Community Gardens; Gardening 
Primary Category: Recreation 
Council: City of Onkaparinga

Org ID: 201870 
Org Name: State Library of SA 
S Suburb: Adelaide 
Org Type: Government 
Local Community dir: Public Libraries 
Primary Category: Communication & Information Services 
Council: City of Adelaide 
Subjects: Audio-Visual Resources

Org ID: 206254 
Org Name: Fleurieu Peninsula Food 
S Suburb: Tooperang  
Services: Representative body for food industry offering membership to food producers and suppliers on the Fleurieu Peninsula, South Australia 
Org Type: Comm

{'input': 'I stay in prospect and i need food',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content="Hi Bob! Welcome to SAcommunity. We're South Australia's community information directory. What brings you here today? \nAre you looking for any particular community service or government aid?"),
  HumanMessage(content="what's my name?"),
  AIMessage(content="Your name is Bob! Hi Bob, it's great to hear from you today. Do you have any queries about a particular organisation or service in South Australia? \n\nPerhaps you could give me some hints about the kind of service you're interested in and I can help point you in the right direction. It would be a pleasure to help you find the information you need."),
  HumanMessage(content='can you remember what I said just now?'),
  AIMessage(content="Yes, you simply said your name was Bob. That's a great name! \n\nCan I help you with any organisation or service searches today? It might be fun to explore what's available in S